## 📦 Step 1: Install Packages

In [ ]:

# !pip install -q diffusers scikit-image

print("✅ Packages ready!")

## 📚 Step 2: Import Libraries

In [ ]:
import os
import numpy as np
import h5py
import torch
import torch.nn as nn
from diffusers import UNet2DModel, DDPMScheduler
import matplotlib.pyplot as plt
from skimage.color import lab2rgb
from tqdm.auto import tqdm

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

print("✅ Imports complete!")

## ⚙️ Step 3: Configuration

In [ ]:
# ============================================================
# PATHS 
# ============================================================
CHECKPOINT_PATH = "C:\\Users\\HP\\Desktop\\Image colorization\\checkpoint_epoch_26.pth" 
VAL_H5_PATH = "C:\\Users\\HP\\Desktop\\Image colorization\\val.h5" 
OUTPUT_DIR = "C:\\Users\\HP\\Desktop\\Image colorization\\inference_results"

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ============================================================
# MODEL PARAMETERS (must match training)
# ============================================================
IMG_SIZE = 128
NUM_INFERENCE_STEPS = 50  # Number of denoising steps
NUM_SAMPLES = 50  # Number of images to colorize

# Diffusion parameters (from training)
NUM_TRAIN_TIMESTEPS = 1000
BETA_START = 0.0001
BETA_END = 0.02
BETA_SCHEDULE = "linear"

print("✅ Configuration set!")

## 🧠 Step 4: Define Model

In [ ]:
class ColorDiffusionModel(nn.Module):
    """Conditional diffusion model for colorization."""
    
    def __init__(self, unet, noise_scheduler):
        super().__init__()
        self.unet = unet
        self.noise_scheduler = noise_scheduler

    def forward(self, L, AB, timesteps):
        """Forward pass: predict noise from L + noisy_AB"""
        model_input = torch.cat([L, AB], dim=1)  # (B, 3, 128, 128)
        predicted_noise = self.unet(model_input, timesteps).sample
        return predicted_noise

    def add_noise(self, AB, noise, timesteps):
        """Add noise to AB channels"""
        return self.noise_scheduler.add_noise(AB, noise, timesteps)

print("✅ Model class defined!")

## 📥 Step 5: Load Model from Checkpoint

In [ ]:
print("Loading model from checkpoint...")

# Initialize UNet (same architecture as training)
unet = UNet2DModel(
    sample_size=IMG_SIZE,
    in_channels=3,
    out_channels=2,
    layers_per_block=2,
    block_out_channels=(96, 192, 384, 512),
    down_block_types=(
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",
    ),
    up_block_types=(
        "AttnUpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
    attention_head_dim=8,
)

# Initialize noise scheduler
noise_scheduler = DDPMScheduler(
    num_train_timesteps=NUM_TRAIN_TIMESTEPS,
    beta_start=BETA_START,
    beta_end=BETA_END,
    beta_schedule=BETA_SCHEDULE,
    prediction_type="epsilon",
)

# Create model
model = ColorDiffusionModel(unet, noise_scheduler)

# Load checkpoint
checkpoint = torch.load(CHECKPOINT_PATH, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

# Print checkpoint info
print(f"✅ Loaded checkpoint from epoch {checkpoint['epoch'] + 1}")
print(f"   Train loss: {checkpoint['train_loss']:.4f}")
print(f"   Val loss: {checkpoint['val_loss']:.4f}")

total_params = sum(p.numel() for p in model.parameters())
print(f"   Model parameters: {total_params:,}")

## 📊 Step 6: Load Validation Data

In [ ]:
print(f"Loading validation data from {VAL_H5_PATH}...")

# Load validation images
with h5py.File(VAL_H5_PATH, 'r') as f:
    # Load first NUM_SAMPLES images
    val_images = f['images'][:NUM_SAMPLES].astype(np.float32)

print(f"✅ Loaded {len(val_images)} validation images")
print(f"   Shape: {val_images.shape}")
print(f"   Dtype: {val_images.dtype}")

# Split into L and AB channels
L_images = val_images[:, :, :, 0:1]  # (N, 128, 128, 1)
AB_images = val_images[:, :, :, 1:3]  # (N, 128, 128, 2)

# Convert to PyTorch tensors: (N, H, W, C) -> (N, C, H, W)
L_batch = torch.from_numpy(L_images).permute(0, 3, 1, 2).to(device)  # (N, 1, 128, 128)
AB_batch_gt = torch.from_numpy(AB_images).permute(0, 3, 1, 2).to(device)  # (N, 2, 128, 128)

print(f"✅ Prepared tensors for inference")
print(f"   L shape: {L_batch.shape}")
print(f"   AB shape: {AB_batch_gt.shape}")

## 🎨 Step 7: Run Inference

In [ ]:
print(f"Running inference with {NUM_INFERENCE_STEPS} denoising steps...")

@torch.no_grad()
def colorize_images(model, L_batch, num_steps=50):
    """
    Colorize grayscale images using diffusion model.
    
    Args:
        model: Trained ColorDiffusionModel
        L_batch: Grayscale images (N, 1, H, W) in range [0, 1]
        num_steps: Number of denoising steps
    
    Returns:
        predicted_AB: Colorized AB channels (N, 2, H, W) in range [-1, 1]
    """
    batch_size = L_batch.shape[0]
    
    # Set timesteps for inference
    model.noise_scheduler.set_timesteps(num_steps)
    
    # Start from random noise
    predicted_AB = torch.randn(batch_size, 2, IMG_SIZE, IMG_SIZE, device=L_batch.device)
    
    # Denoise step by step
    for t in tqdm(model.noise_scheduler.timesteps, desc="Denoising"):
        # Create timestep tensor
        timesteps = torch.full((batch_size,), t, device=L_batch.device, dtype=torch.long)
        
        # Predict noise
        pred_noise = model(L_batch, predicted_AB, timesteps)
        
        # Denoise
        predicted_AB = model.noise_scheduler.step(pred_noise, t, predicted_AB).prev_sample
    
    # Clamp to valid range
    predicted_AB = predicted_AB.clamp(-1.0, 1.0)
    
    return predicted_AB

# Run inference
predicted_AB = colorize_images(model, L_batch, num_steps=NUM_INFERENCE_STEPS)

print(f"✅ Inference complete!")
print(f"   Predicted AB range: [{predicted_AB.min().item():.3f}, {predicted_AB.max().item():.3f}]")

## 📊 Step 8: Display Results

In [ ]:
# Move to CPU for visualization
L_batch_cpu = L_batch.cpu()
AB_batch_gt_cpu = AB_batch_gt.cpu()
predicted_AB_cpu = predicted_AB.cpu()

# Function to convert LAB to RGB
def lab_to_rgb(L, AB):
    """
    Convert LAB tensors to RGB numpy array.
    
    Args:
        L: (C, H, W) tensor in range [0, 1]
        AB: (C, H, W) tensor in range [-1, 1]
    
    Returns:
        RGB image (H, W, 3) in range [0, 1]
    """
    # Concatenate L and AB
    lab = torch.cat([L, AB], dim=0)  # (3, H, W)
    lab = lab.permute(1, 2, 0).numpy()  # (H, W, 3)
    
    # Denormalize: L: [0,1]->[ 0,100], AB: [-1,1]->[-128,128]
    lab = np.stack([
        lab[:, :, 0] * 100,
        lab[:, :, 1] * 128,
        lab[:, :, 2] * 128
    ], axis=-1)
    
    # Convert to RGB
    rgb = lab2rgb(lab)
    return np.clip(rgb, 0, 1)

# Display results
num_display = min(8, NUM_SAMPLES)
fig, axes = plt.subplots(3, num_display, figsize=(num_display * 3, 9))

for i in range(num_display):
    # Grayscale input
    L_gray = L_batch_cpu[i, 0].numpy()
    axes[0, i].imshow(L_gray, cmap='gray')
    axes[0, i].set_title('Input (Grayscale)', fontweight='bold')
    axes[0, i].axis('off')

    # Ground truth
    rgb_gt = lab_to_rgb(L_batch_cpu[i], AB_batch_gt_cpu[i])
    axes[1, i].imshow(rgb_gt)
    axes[1, i].set_title('Ground Truth', fontweight='bold')
    axes[1, i].axis('off')

    # Prediction
    rgb_pred = lab_to_rgb(L_batch_cpu[i], predicted_AB_cpu[i])
    axes[2, i].imshow(rgb_pred)
    axes[2, i].set_title('Prediction', fontweight='bold')
    axes[2, i].axis('off')

plt.tight_layout()
output_path = os.path.join(OUTPUT_DIR, 'inference_results.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"✅ Saved results to {output_path}")
plt.show()

## 📈 Step 9: Compute Metrics

In [ ]:
print("Computing metrics...")

# Compute statistics
with torch.no_grad():
    # Ground truth statistics
    gt_mean_a = AB_batch_gt_cpu[:, 0].mean().item()
    gt_mean_b = AB_batch_gt_cpu[:, 1].mean().item()
    gt_std_a = AB_batch_gt_cpu[:, 0].std().item()
    gt_std_b = AB_batch_gt_cpu[:, 1].std().item()
    
    # Prediction statistics
    pred_mean_a = predicted_AB_cpu[:, 0].mean().item()
    pred_mean_b = predicted_AB_cpu[:, 1].mean().item()
    pred_std_a = predicted_AB_cpu[:, 0].std().item()
    pred_std_b = predicted_AB_cpu[:, 1].std().item()
    
    # MSE between prediction and ground truth
    mse = ((predicted_AB_cpu - AB_batch_gt_cpu) ** 2).mean().item()

print("\n" + "=" * 70)
print("📊 INFERENCE METRICS")
print("=" * 70)
print(f"\nGround Truth:")
print(f"  A channel: mean = {gt_mean_a:+.3f}, std = {gt_std_a:.3f}")
print(f"  B channel: mean = {gt_mean_b:+.3f}, std = {gt_std_b:.3f}")
print(f"\nPrediction:")
print(f"  A channel: mean = {pred_mean_a:+.3f}, std = {pred_std_a:.3f}")
print(f"  B channel: mean = {pred_mean_b:+.3f}, std = {pred_std_b:.3f}")
print(f"\nError:")
print(f"  MSE (AB channels): {mse:.4f}")
print(f"\nSaturation (std ratio):")
print(f"  A channel: {pred_std_a / gt_std_a * 100:.1f}%")
print(f"  B channel: {pred_std_b / gt_std_b * 100:.1f}%")
print("=" * 70)

## 💾 Step 10: Save Individual Results 

In [ ]:
# Save individual images
print("Saving individual results...")

for i in range(NUM_SAMPLES):
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    # Grayscale
    L_gray = L_batch_cpu[i, 0].numpy()
    axes[0].imshow(L_gray, cmap='gray')
    axes[0].set_title('Input', fontweight='bold')
    axes[0].axis('off')
    
    # Ground truth
    rgb_gt = lab_to_rgb(L_batch_cpu[i], AB_batch_gt_cpu[i])
    axes[1].imshow(rgb_gt)
    axes[1].set_title('Ground Truth', fontweight='bold')
    axes[1].axis('off')
    
    # Prediction
    rgb_pred = lab_to_rgb(L_batch_cpu[i], predicted_AB_cpu[i])
    axes[2].imshow(rgb_pred)
    axes[2].set_title('Prediction', fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    save_path = os.path.join(OUTPUT_DIR, f'result_{i+1:02d}.png')
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

print(f"✅ Saved {NUM_SAMPLES} individual results to {OUTPUT_DIR}")

## 🔍 Step 11: Upscale Results 

Upscale colorized images from 128×128 to higher resolution for better viewing quality.

In [ ]:
from PIL import Image, ImageFilter
from skimage.color import rgb2lab

# Configuration
UPSCALE_SIZE = 1024  # Target resolution (1024x1024)
NUM_UPSCALE_SAMPLES = min(8, NUM_SAMPLES)  # How many to upscale and save

print(f"Upscaling {NUM_UPSCALE_SAMPLES} images from 128×128 to {UPSCALE_SIZE}×{UPSCALE_SIZE}...")
print("Using smart upscaling: preserving full-resolution L channel + upscaling AB only")

# Create upscaled directory
upscaled_dir = os.path.join(OUTPUT_DIR, "upscaled")
os.makedirs(upscaled_dir, exist_ok=True)

def smart_upscale(L_tensor_128, predicted_AB_128, target_size=1024):
    """
    Smart upscaling that preserves detail by:
    1. Keeping full-resolution L channel from original 128x128
    2. Upscaling only AB channels using high-quality interpolation
    3. Combining full-res L with upscaled AB
    
    Args:
        L_tensor_128: (1, 128, 128) L channel normalized [0, 1]
        predicted_AB_128: (2, 128, 128) AB channels normalized [-1, 1]
        target_size: Output resolution
    
    Returns:
        RGB image as PIL Image at target_size × target_size
    """
    # Get L channel at 128x128 (we'll upscale this too for consistency)
    L_128 = L_tensor_128.numpy() * 100.0  # Denormalize to [0, 100]
    
    # Get predicted AB at 128x128
    predicted_AB = predicted_AB_128.numpy()  # (2, 128, 128)
    predicted_AB = predicted_AB.transpose(1, 2, 0)  # (128, 128, 2)
    
    # Denormalize AB channels: [-1, 1] -> [-128, 128]
    predicted_AB_denorm = predicted_AB * 128.0
    
    # Upscale L channel using high-quality LANCZOS interpolation
    L_pil = Image.fromarray(L_128[0].astype(np.float32), mode='F')
    L_upscaled = L_pil.resize((target_size, target_size), Image.Resampling.LANCZOS)
    L_upscaled_array = np.array(L_upscaled)
    
    # Upscale AB channels separately using LANCZOS
    a_channel_pil = Image.fromarray(predicted_AB_denorm[:, :, 0].astype(np.float32), mode='F')
    b_channel_pil = Image.fromarray(predicted_AB_denorm[:, :, 1].astype(np.float32), mode='F')
    
    a_channel_upscaled = a_channel_pil.resize((target_size, target_size), Image.Resampling.LANCZOS)
    b_channel_upscaled = b_channel_pil.resize((target_size, target_size), Image.Resampling.LANCZOS)
    
    a_channel_full = np.array(a_channel_upscaled)
    b_channel_full = np.array(b_channel_upscaled)
    
    # Combine full-resolution L with upscaled AB
    result_lab = np.zeros((target_size, target_size, 3), dtype=np.float64)
    result_lab[:, :, 0] = L_upscaled_array  # Upscaled L channel
    result_lab[:, :, 1] = a_channel_full     # Upscaled A channel
    result_lab[:, :, 2] = b_channel_full     # Upscaled B channel
    
    # Convert LAB to RGB
    result_rgb = lab2rgb(result_lab)
    result_rgb = np.clip(result_rgb, 0, 1)
    
    # Convert to PIL Image
    result_pil = Image.fromarray((result_rgb * 255).astype(np.uint8))
    
    return result_pil

# Upscale and save results
fig, axes = plt.subplots(3, NUM_UPSCALE_SAMPLES, figsize=(NUM_UPSCALE_SAMPLES * 3, 9))

for i in range(NUM_UPSCALE_SAMPLES):
    # Grayscale upscaled
    L_gray = L_batch_cpu[i, 0].numpy()
    L_upscaled_pil = Image.fromarray((L_gray * 255).astype(np.uint8), mode='L')
    L_upscaled_pil = L_upscaled_pil.resize((UPSCALE_SIZE, UPSCALE_SIZE), Image.Resampling.LANCZOS)
    
    axes[0, i].imshow(np.array(L_upscaled_pil), cmap='gray')
    axes[0, i].set_title(f'Input {UPSCALE_SIZE}×{UPSCALE_SIZE}', fontweight='bold', fontsize=9)
    axes[0, i].axis('off')
    
    # Ground truth - smart upscale
    gt_upscaled = smart_upscale(L_batch_cpu[i], AB_batch_gt_cpu[i], UPSCALE_SIZE)
    axes[1, i].imshow(gt_upscaled)
    axes[1, i].set_title(f'GT {UPSCALE_SIZE}×{UPSCALE_SIZE}', fontweight='bold', fontsize=9)
    axes[1, i].axis('off')
    
    # Prediction - smart upscale
    pred_upscaled = smart_upscale(L_batch_cpu[i], predicted_AB_cpu[i], UPSCALE_SIZE)
    axes[2, i].imshow(pred_upscaled)
    axes[2, i].set_title(f'Predicted {UPSCALE_SIZE}×{UPSCALE_SIZE}', fontweight='bold', fontsize=9)
    axes[2, i].axis('off')
    
    # Save individual upscaled images
    L_upscaled_pil.save(os.path.join(upscaled_dir, f'upscaled_{i+1:02d}_input.png'))
    gt_upscaled.save(os.path.join(upscaled_dir, f'upscaled_{i+1:02d}_groundtruth.png'))
    pred_upscaled.save(os.path.join(upscaled_dir, f'upscaled_{i+1:02d}_prediction.png'))

plt.tight_layout()
comparison_path = os.path.join(upscaled_dir, 'upscaled_comparison.png')
plt.savefig(comparison_path, dpi=150, bbox_inches='tight')
print(f"✅ Saved upscaled comparison to {comparison_path}")
print(f"✅ Saved {NUM_UPSCALE_SAMPLES * 3} individual upscaled images to {upscaled_dir}/")
plt.show()

print(f"\n{'='*70}")
print(f"📊 Upscaling Summary:")
print(f"  Original: 128×128 pixels")
print(f"  Upscaled: {UPSCALE_SIZE}×{UPSCALE_SIZE} pixels (8x resolution)")
print(f"  Method: Smart upscaling (LANCZOS interpolation on L and AB separately)")
print(f"  Detail preservation: Full-resolution L channel + upscaled AB channels")
print(f"  Output: {upscaled_dir}/")
print(f"{'='*70}")